In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from backtesting import Backtest, Strategy
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


Loading BokehJS ...

In [2]:
#Retrieve pair price history from Yahoo Finance using pandas_ta
df = pd.DataFrame()
df = df.ta.ticker("GBPUSD=X")
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2003-12-01 00:00:00+00:00,1.723811,1.727205,1.718390,1.718597,0,0.0,0.0
2003-12-02 00:00:00+00:00,1.719010,1.732112,1.717298,1.730313,0,0.0,0.0
2003-12-03 00:00:00+00:00,1.730493,1.731812,1.725209,1.728101,0,0.0,0.0
2003-12-04 00:00:00+00:00,1.727414,1.728997,1.718302,1.720697,0,0.0,0.0
2003-12-05 00:00:00+00:00,1.720608,1.733102,1.719809,1.733102,0,0.0,0.0
...,...,...,...,...,...,...,...
2023-05-02 00:00:00+01:00,1.248736,1.251220,1.243657,1.248595,0,0.0,0.0
2023-05-03 00:00:00+01:00,1.247972,1.255209,1.246961,1.247661,0,0.0,0.0
2023-05-04 00:00:00+01:00,1.256866,1.259684,1.255225,1.256897,0,0.0,0.0


In [3]:
#Drop uneccassary columns
del df["Dividends"]
del df["Stock Splits"]
del df["Volume"]

In [4]:
#Create new column indicating next day close price
df["Tomorrow_Close"] = df["Close"].shift(-1)

In [5]:
#Used as a target variable to see if model correctly predicts price action
df["Target"] = (df["Tomorrow_Close"] > df["Close"]).astype(int)


In [6]:

time_horizons = [2,5,60,250,1000]
new_features = []

for h in time_horizons:
    #calculates the rolling average for each of the respective time horizons provided
    rolling_averages = df.rolling(h).mean()
    
    ratio = f"Close_Ratio_{h}"
    df[ratio] = df["Close"] / rolling_averages["Close"]
    
    #Trend is the number of days within the time horizon, where the price went up 
    trend = f"Trend_{h}"
    df[trend] = df.shift(1).rolling(h).sum()["Target"]
    
    new_features+= [ratio, trend]
    

In [7]:
#Adding RSI column with 14 day lookback as it is the standard
df['RSI'] = ta.rsi(close = df["Close"], length= 14)
new_features.append("RSI")

In [8]:
#Adding MACD indicator columns with 12 and 26 day ema and 9 unit period ema of macd chart
macd = ta.macd(close = df['Close'], fast = 12, slow = 26, signal = 9)
df= pd.concat([df, macd], axis = 1)
new_features+= list(macd)

In [9]:
#Adding Stochastic Oscillator indicator
stoch = ta.stoch(high = df['High'], low = df['Low'], close = df['Close'], k= 14) 
df = pd.concat([df,stoch], axis = 1)
new_features+= list(stoch)

In [10]:
#Drop null values created by indicator creation
df = df.dropna()
df

,Open,High,Low,Close,Tomorrow_Close,Target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,...,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000,RSI,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,STOCHk_14_3_3,STOCHd_14_3_3
Date,,,,,,,,,,,,,,,,,,,,,
2007-10-04 00:00:00+01:00,2.032603,2.042400,2.028315,2.038611,2.043694,1,1.001476,1.0,0.999151,2.0,...,1.031575,132.0,1.095111,507.0,57.885143,0.006289,0.002464,0.003825,80.072541,84.726310
2007-10-05 00:00:00+01:00,2.038487,2.043820,2.031901,2.043694,2.035416,0,1.001245,2.0,1.001888,2.0,...,1.033799,132.0,1.097656,507.0,59.962376,0.006940,0.002492,0.004448,82.107199,81.794322
2007-10-08 00:00:00+01:00,2.041900,2.042901,2.033595,2.035416,2.037697,1,0.997971,1.0,0.998702,2.0,...,1.029294,131.0,1.093030,507.0,55.188454,0.006710,0.001810,0.004900,82.568243,81.582661
2007-10-09 00:00:00+01:00,2.035499,2.038985,2.026301,2.037697,2.041816,1,1.000560,1.0,1.000046,3.0,...,1.030106,132.0,1.094069,508.0,56.222705,0.006635,0.001388,0.005247,80.884388,81.853277
2007-10-10 00:00:00+01:00,2.037697,2.047209,2.037697,2.041816,2.034215,0,1.001010,2.0,1.001162,4.0,...,1.031837,133.0,1.096098,508.0,58.103188,0.006829,0.001266,0.005564,77.813756,80.422129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-01 00:00:00+01:00,1.255840,1.257000,1.248954,1.255761,1.248595,0,1.002399,2.0,1.005552,4.0,...,1.046120,132.0,0.975436,498.0,62.502240,0.006112,-0.000188,0.006300,71.463313,60.450082
2023-05-02 00:00:00+01:00,1.248736,1.251220,1.243657,1.248595,1.247661,0,0.997139,1.0,1.000039,3.0,...,1.040095,131.0,0.969882,498.0,56.169330,0.005834,-0.000373,0.006207,70.122656,64.770662
2023-05-03 00:00:00+01:00,1.247972,1.255209,1.246961,1.247661,1.256897,1,0.999626,0.0,0.998302,3.0,...,1.039319,130.0,0.969165,498.0,55.381028,0.005475,-0.000585,0.006061,67.045379,69.543783


In [11]:
df["Target"].value_counts()

Target
0    2059
1    1994
Name: count, dtype: int64

In [12]:
#Splitting dataset in to training and testing sets to compare base model and tuned model 

X = df[new_features]  
y = df["Target"]

train_num = int(0.8*len(X))

x_train, x_test = X[:train_num], X[train_num:]
y_train, y_test = y[:train_num], y[train_num:]

x_train.shape, x_test.shape, 

((3242, 16), (811, 16))

In [13]:
#Hyperparameter tuning for Random Forest
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [14]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train, y_train)

Fitting 2 folds for each of 100 candidates, totalling 200 fits


RandomizedSearchCV(cv=2, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [15]:
# Best configuration of parameters
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_depth': 70,
 'bootstrap': True}

In [16]:
#Average classification accuracy across 10 folds
n_folds = 10
best_model = rf_random.best_estimator_
cv_error = np.average(cross_val_score(best_model, x_train, y_train, cv=n_folds))
print('The {}-fold cross-validation accuracy score for this classifier is {:.2f}'.format(n_folds, cv_error))

The 10-fold cross-validation accuracy score for this classifier is 0.47


In [17]:
all_predictions = []
for _ in range(10):
    predictions = best_model.predict(x_test)
    all_predictions.append(predictions)
    
average_pred = sum(all_predictions) / len(all_predictions)
final_pred = np.where(average_pred >= 0.5, 1, 0)

print(classification_report(y_test, final_pred))

              precision    recall  f1-score   support

           0       0.52      0.67      0.58       396
           1       0.56      0.40      0.47       415

    accuracy                           0.53       811
   macro avg       0.54      0.54      0.53       811
weighted avg       0.54      0.53      0.52       811



In [18]:
#Trading strategy incorperating Random Forest

#Use 80% of data to train model 
training_data = round(0.8*len(df))

class RandomForestStrategy(Strategy):
    delta = 0.004
    
    def init(self):
        
        #Initialise model using best hyper parameters
        self.model = rf_random.best_estimator_
        
        #Train model using training set
        df = self.data.df.iloc[:training_data]
        self.model.fit(df[new_features], df["Target"])
        
        #Create custom indicator for tmr predictions
        self.predictions = self.I(lambda: np.repeat(np.nan, len(self.data)), name = "prediction")
        
    def next(self):
        
        #Skip the data used for training and only perform trades on testing set
        if len(self.data) < training_data:
            return
        
        #Prepare variables for take profit and stop loss calculations 
        high, low, close = self.data.High, self.data.Low, self.data.Close
        
        #Predict the trend for the next close candle (upwards or downwards)
        x = self.data.df.iloc[-1:]
        prediction = self.model.predict(x[new_features])[0]
  
        #Update the predictions indicator 
        self.predictions[-1] = prediction
        
        
        #upper is the take profit which is calculated to be one price delta away from closing price
        #lower is the stop loss which is calculated to be one price delta away from the closing price 
        upper,lower = close[-1] * (1 + np.r_[1, -1] * self.delta)
        
        #If the prediction is upwards and a long position is not held, place a long order using 20% of the accounts equity 
        #if the prediction is downwards and a short position is not help, place a short order using 20% of the accounts equity
        if prediction == 1 and not self.position.is_long:
            self.buy(size =0.2, tp = upper, sl=lower)
        elif prediction == 0 and not self.position.is_short:
            self.sell(size =0.2, tp=lower, sl=upper)
            
            
#Single backtest run using RandomForestStrategy
backtest = Backtest(df, RandomForestStrategy, margin= 0.1)
backtest.run()

Start                     2007-10-04 00:00...
End                       2023-05-05 00:00...
Duration                   5692 days 00:00:00
Exposure Time [%]                   20.009869
Equity Final [$]                   6371.77613
Equity Peak [$]                  10015.410437
Return [%]                         -36.282239
Buy & Hold Return [%]               -38.27482
Return (Ann.) [%]                   -2.763421
Volatility (Ann.) [%]                3.661013
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -36.380279
Avg. Drawdown [%]                  -36.380279
Max. Drawdown Duration     1129 days 00:00:00
Avg. Drawdown Duration     1129 days 00:00:00
# Trades                                  738
Win Rate [%]                        26.151762
Best Trade [%]                       1.067025
Worst Trade [%]                     -2.021528
Avg. Trade [%]                    

In [19]:
#Configure backtest with dataset, trading strategy and margin
backtest = Backtest(df, RandomForestStrategy, margin= 0.1)

results = np.zeros((5, 6))

for i in range(5):
    stats = backtest.run()
    results[i] = [stats['Return [%]'], stats['Win Rate [%]'], stats['Max. Drawdown [%]'],
                  stats['Avg. Drawdown [%]'], stats['Equity Final [$]'], stats['Equity Peak [$]']]

# Calculate the average statistics for 5 runs
averager = np.mean(results[:, 0])
averageb = np.mean(results[:, 1])
am_dd = np.mean(results[:, 2])
aa_dd = np.mean(results[:, 3])
eqf = np.mean(results[:, 4])
eqp = np.mean(results[:, 5])

# Print the average statistics
print("Mean Return%: {}".format(averager))
print("Mean Win Rate%: {}".format(averageb))
print("Mean Final Equity$: {}".format(eqf))
print("Mean Peak Equity$: {}".format(eqp))
print("Mean Max Drawdown%: {}".format(am_dd))
print("Mean Average Drawdown%: {}".format(aa_dd))


Mean Return%: -34.088742130529525
Mean Win Rate%: 26.194996960703328
Mean Final Equity$: 6591.125786947047
Mean Peak Equity$: 10064.600011970806
Mean Max Drawdown%: -34.51118796172749
Mean Average Drawdown%: -24.571697650797887
